In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras import utils
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import pathlib
import zipfile
import PIL
from PIL import Image
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from tensorflow.keras.metrics import Precision, Recall, AUC
from sklearn.metrics import ConfusionMatrixDisplay
from tensorflow.keras.callbacks import EarlyStopping
from pathlib import Path
from keras.preprocessing import image

#from google.colab import drive
#drive.mount('/content/gdrive')

# Set up Images in Datasets
def extractDataSet(folder_path, labels, label_mode, image_size, batch_size, seed, shuffle, interpolation):
    return tf.keras.utils.image_dataset_from_directory(
        directory=folder_path,
        labels=labels,
        label_mode=label_mode,
        image_size=image_size,
        batch_size=batch_size,
        interpolation=interpolation,
        seed=seed,
        shuffle=shuffle
    )

# Simple Model
def simpleModel():
      return Sequential([
          #Convolution block #1
          #Setting up convolution block
          layers.Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 3)),
          #Adding Max Pooling
          layers.MaxPooling2D((2, 2)),
          #Setting up convolution block
          layers.Conv2D(32, (3, 3), activation='relu'),
          #Adding Max Pooling layers.MaxPooling2D((2, 2)),
          #Transition to Classification
          layers.Flatten(),
          #DNN layers.Dense(128, activation='relu'),
          layers.Dense(5, activation='softmax')
      ])

# Print Accuracy And Loss
def printAccuracyAndLoss(history, datasetName):
    if datasetName == "Training":
        print(f"{datasetName} Accuracy: {history.history['accuracy'][-1]:.2f}")
        print(f"{datasetName} Loss: {history.history['loss'][-1]:.2f}")
    elif datasetName == "Validation":
        print(f"{datasetName} Accuracy: {history.history['val_accuracy'][-1]:.2f}")
        print(f"{datasetName} Loss: {history.history['val_loss'][-1]:.2f}")


# Plot Image
def plotMetrics(epochs_range, training_metric, val_metric, training_label, val_label, title):
    plt.figure(figsize=(8, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, training_metric, label=training_label)
    plt.plot(epochs_range, val_metric, label=val_label)
    plt.legend(loc='upper right')
    plt.title(title)

# Diplay Image
def display_image(image_path):
    img = Image.open(str(image_path))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

#Predict Image
def predict_image(image_path):
    test_image = image.load_img(image_path, target_size = (512, 512))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = np.array(model.predict(test_image))
    print(result)
    if result[0][0] == 0:
        prediction = '0'
    elif result[0][0] == 1:
        prediction = '1'
    elif result[0][0] == 2:
        prediction = '2'
    elif result[0][0] == 3:
        prediction = '3'
    elif result[0][0] == 4:
        prediction = '4'
    else:
        prediction = '0'

    print(prediction)

# Print Predicted vs. True Class Labels along with image
def printPredictedAndTrueClassLabels(test_images, test_labels, model):
    test_images_np = np.array(test_images)
    #Predict image labels with provided model
    predictions = model.predict(test_images_np, verbose=0)
    print(predictions)
    #Identify image labels from predictions and print
    predicted_classes = np.argmax(predictions, axis=1)
    print(predicted_classes)
    #Identify true labels and print
    true_classes = np.array(test_labels)
    print(true_classes)
    return true_classes, predicted_classes

# Print Confusion Matrix and Classification Report
def printConfusionMatrixAndClassificationReport(true_classes, predicted_classes):
    # Classification Report
    print('# Classification Report')
    print(classification_report(true_classes, predicted_classes))
    # Confusion Matrix
    conf_matrix = confusion_matrix(true_classes, predicted_classes)
    print("Confusion Matrix:")
    print(conf_matrix)
    # Heatmap visualization for confusion matrix
    cm_display = ConfusionMatrixDisplay(confusion_matrix = conf_matrix, display_labels = ["0", "1", "2", "3", "4"])
    # Display matrix
    cm_display.plot()
    # Save matrix to file
    filename = f"CNN_Simple_Model_Confusion_matrix_v1.png"
    full_path = os.path.join(DATA_PIPELINE_PATH, filename)

    print(f"  CNN Simple Model Confusion Matrix v1: {filename}")
    print(f"Images saved to: {DATA_PIPELINE_PATH}")
    plt.tight_layout()
    cm_display.figure_.savefig(full_path, bbox_inches='tight')

    # --- Pipeline Begin ---

if __name__ == "__main__":

    # Data Path
    DATA_PIPELINE_PATH = '/data/processed/'
    #DATA_PIPELINE_PATH = Path('/content/gdrive/MyDrive/AIML-Bootcamp/Colab_Data/Data_Pipeline')

    # Set dataset parameters
    image_size = (512, 512)
    img_height = 512
    img_width = 512
    batch_size = 32
    seed = 0
    validation_split_ratio = 0 # 0% for validation, since the images are already split into train, test and val
    interpolation="bilinear"
    labels="inferred"
    label_mode='int'
    shuffle=True

    # Load datasets
    train_ds = extractDataSet(DATA_PIPELINE_PATH / 'train', labels, label_mode, image_size, batch_size, seed, True, interpolation)
    test_ds = extractDataSet(DATA_PIPELINE_PATH / 'test', labels, label_mode, image_size, batch_size, seed, True, interpolation)
    val_ds = extractDataSet(DATA_PIPELINE_PATH / 'val', labels, label_mode, image_size, batch_size, seed, False, interpolation)

    # Simple Model
    model = simpleModel()
    model.summary()

    # Compile Model
    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

    # Fit Model
    epochs = 10
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs
    )

    # Print the training and validation accuracies
    printAccuracyAndLoss(history, "Training")
    printAccuracyAndLoss(history, "Validation")

    epochs_range = range(epochs)

    # Plot Training and Validation Accuracy
    plotMetrics(epochs_range, history.history['accuracy'], history.history['val_accuracy'], 'Training Accuracy', 'Validation Accuracy', 'Training and Validation Accuracy')

    # Plot Training and Validation Loss
    plotMetrics(epochs_range, history.history['loss'], history.history['val_loss'], 'Training Loss', 'Validation Loss', 'Training and Validation Loss')

    # Build Test images and labels
    test_images = []
    test_labels = []
    for images, labels in test_ds:
        test_images.extend(images.numpy())
        test_labels.extend(labels.numpy())

    # Calculate test accuracy
    test_loss, test_accuracy = model.evaluate(test_ds, verbose=0)
    print(f"Test Accuracy: {test_accuracy:.3f} ({test_accuracy*100:.1f}%)")
    print(f"Test Loss: {test_loss:.3f} ({test_loss:.1f}%)")

    # Display some images
    # Get file paths from the test directory structure
    test_image_files_list = []
    test_extract_dir = DATA_PIPELINE_PATH / 'test'
    test_image_extensions = ("*/*.jpg", "*/*.jpeg", "*/*.png", "*/*.gif", "*/*.bmp", "*/*.tiff")

    for ext in test_image_extensions:
        test_image_files_list.extend(glob.glob(os.path.join(test_extract_dir, ext)))

    for i in range(30):
      print(i)
      display_image(test_image_files_list[i])
      predict_image(test_image_files_list[i])

    # Get predictions for test images
    true_classes, predicted_classes = printPredictedAndTrueClassLabels(test_images, test_labels, model)

    # Print Confusion Matrix And Classification Report
    printConfusionMatrixAndClassificationReport(true_classes, predicted_classes)